# ZBP to ZCTA
Retrieves data from the Census Bureau's ZIP Code Business Patterns API for a specific area and summarizes it by ZCTA. Three tables are generated: one for employees that contains employment, establishments, and wages, one for industries that contains counts of establishments by 2-digit sector NAICS codes, and one reference table that correlates sector numbers and names. Initial data retrieved from API is written to json, final output is written to a SQLite database. 

Confirmed to work with the 2021 ZBP series

https://www.census.gov/data/developers/data-sets/cbp-zbp/cbp-api.html

## Variables

In [3]:
import pandas as pd, requests, sqlite3, os, json, time
from IPython.display import clear_output

In [7]:
#Crosswalk file from UDS Mapper
uszips_file='ZIPCodetoZCTACrosswalk2022UDS.xlsx'
state_abrev='RI' #State with ZIPs we want to pull

uszips_path=os.path.join('inputs',uszips_file)

#Dump files for api data storage
ejsonpath=os.path.join('outputs', 'emp_data.json')
ijsonpath=os.path.join('outputs', 'ind_data.json')
cjsonpath=os.path.join('outputs', 'codes_data.json')

#API variables - UPDATE THE YEAR
keyfile='census_key.txt'

year='2022'
dsource='cbp'
state='44'
ecols='ESTAB,EMP,PAYQTR1,PAYANN'
icols='ESTAB'
ncodes=['00','11','21','22','23','31-33','42','44-45','48-49',
        '51','52','53','54','55','56','61','62','71','72','81',
        '99']

#SQL output - UPDATE EACH TABLE NAME
dbname=os.path.join('outputs','testdb.sqlite')
emptable='zbp2022_emp'
indtable='zbp2022_ind'
codetable='zbp2022_indcodes'
cwalktable='zip2zcta_2022'

## Crosswalking
Read in the file that relates US ZIP codes to ZCTAs (from UDS Mapper) and
pull out codes for specific state

In [10]:
uszips=pd.read_excel(uszips_path,dtype={'ZIP_CODE': str, 'zcta': str})
uszips.head()

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type
0,00501,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
1,00544,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
2,00601,Adjuntas,PR,Zip Code Area,00601,Zip matches ZCTA
3,00602,Aguada,PR,Zip Code Area,00602,Zip matches ZCTA
4,00603,Aguadilla,PR,Zip Code Area,00603,Zip matches ZCTA


In [12]:
#All ZIP Codes in US
uszips.shape

(41061, 6)

In [14]:
statezips=uszips.loc[uszips['STATE']==state_abrev].set_index('ZIP_CODE')
statezips.head()

,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type
ZIP_CODE,,,,,
02801,Adamsville,RI,Post Office or large volume customer,02837,Spatial join to ZCTA
02802,Albion,RI,Zip Code Area,02802,Zip matches ZCTA
02804,Ashaway,RI,Zip Code Area,02804,Zip matches ZCTA
02806,Barrington,RI,Zip Code Area,02806,Zip matches ZCTA
02807,Block Island,RI,Zip Code Area,02807,Zip matches ZCTA


In [16]:
#ZIPS in local area
statezips.shape

(90, 5)

## API Call and Processing
Request the ZBP data from the Census Bureau for the state, join the ZBP data to the local ZCTA to ZIP table based on ZIP Code, and group the data by ZCTA 

In [19]:
with open(keyfile) as key:
    api_key=key.read().strip()

In [21]:
base_url = f'https://api.census.gov/data/{year}/{dsource}'
base_url

'https://api.census.gov/data/2022/cbp'

### ZBP Employment Data
This data is requested in a series of chunks which contain multiple ZIP Codes - do not rerun the requests block if retrieval is successful but subsequent notebook changes are needed. Proceed to the next block and pull data from json dump file.

In [24]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [26]:
reqzips=list(chunks(statezips.index.tolist(),48))
print('Number of chunks:',len(reqzips))

Number of chunks: 2


#### ***THIS BLOCK IS A REQUESTS BLOCK!*** 

In [29]:
#Code 200 = success, do not rerun this block unless it's necessary
emp_data=[]
for i, v in enumerate (reqzips):
    batchzips=','.join(v)
    edata_url = f'{base_url}?get={ecols}&EMPSZES=001&for=zip code:{batchzips}&key={api_key}'
    response=requests.get(edata_url)
    if response.status_code==200:
        clear_output(wait=True)
        data=response.json()
        if i == 0:    
            for record in data:
                emp_data.append(record)
        else:
            for record in data[1:]:
                emp_data.append(record) 
        print('Retrieved data for chunk',i)
    else:
        print('***Problem with retrieval***, response code',response.status_code)
        break
with open(ejsonpath, 'w') as f:
    json.dump(emp_data, f)
print('Done - Data dumped to json file')

Retrieved data for chunk 1
Done - Data dumped to json file


In [31]:
with open(ejsonpath, 'r') as f:
    ejsondata=json.load(f)
zbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0]).rename(columns={'PAYQTR1':'PAYQ1','PAYANN':'PAYAN','zip code':'zipcode'}).set_index('zipcode')
for field in zbpemp.columns:
    zbpemp=zbpemp.astype(dtype={field:'int64'})
zbpemp.drop(columns=['EMPSZES'],inplace=True)
zbpemp.head()

,ESTAB,EMP,PAYQ1,PAYAN
zipcode,,,,
02801,6,21,732,3089
02802,9,24,198,1747
02804,83,689,5872,28243
02806,361,4069,33814,154223
02808,27,116,938,4414


In [33]:
#ZIP Codes retrieved - may differ from statezips as some zips have no businesses
zbpemp.shape

(87, 4)

In [35]:
# Checking what those missing ZIPs are, for which there is no ZBP data
statezips.index.difference(zbpemp.index)

Index(['02823', '02836', '02877'], dtype='object')

In [37]:
#Flag columns count the number of establishments for which data is not disclosed
flags=['FLAG_EMP','FLAG_PAYQ1','FLAG_PAYAN']
for flagcol in flags:
    datacol=flagcol.split('_')[1]
    zbpemp[flagcol]=0
    zbpemp.loc[zbpemp[datacol] == 0, flagcol] = zbpemp['ESTAB']
zbpemp.head()

,ESTAB,EMP,PAYQ1,PAYAN,FLAG_EMP,FLAG_PAYQ1,FLAG_PAYAN
zipcode,,,,,,,
02801,6,21,732,3089,0,0,0
02802,9,24,198,1747,0,0,0
02804,83,689,5872,28243,0,0,0
02806,361,4069,33814,154223,0,0,0
02808,27,116,938,4414,0,0,0


In [39]:
#Join to ZIP ZCTA crosswalk
zbpemp2zcta = pd.merge(statezips,zbpemp,how='inner',left_index=True,right_index=True)
zbpemp2zcta.index.name = 'ZIP_CODE'
zbpemp2zcta.head()

,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type,ESTAB,EMP,PAYQ1,PAYAN,FLAG_EMP,FLAG_PAYQ1,FLAG_PAYAN
ZIP_CODE,,,,,,,,,,,,
02801,Adamsville,RI,Post Office or large volume customer,02837,Spatial join to ZCTA,6,21,732,3089,0,0,0
02802,Albion,RI,Zip Code Area,02802,Zip matches ZCTA,9,24,198,1747,0,0,0
02804,Ashaway,RI,Zip Code Area,02804,Zip matches ZCTA,83,689,5872,28243,0,0,0
02806,Barrington,RI,Zip Code Area,02806,Zip matches ZCTA,361,4069,33814,154223,0,0,0
02807,Block Island,RI,Zip Code Area,02807,Zip matches ZCTA,189,382,3889,42693,0,0,0


In [41]:
#ZIP count
zbpemp2zcta.shape

(87, 12)

In [43]:
#Aggregate to ZCTAs
zctaemp=zbpemp2zcta[['zcta','ESTAB','EMP','PAYQ1','PAYAN','FLAG_EMP','FLAG_PAYQ1','FLAG_PAYAN']].groupby(['zcta'])[['ESTAB','EMP','PAYQ1','PAYAN','FLAG_EMP','FLAG_PAYQ1','FLAG_PAYAN']].sum()
zctaemp.head()

,ESTAB,EMP,PAYQ1,PAYAN,FLAG_EMP,FLAG_PAYQ1,FLAG_PAYAN
zcta,,,,,,,
02802,9,24,198,1747,0,0,0
02804,83,689,5872,28243,0,0,0
02806,361,4069,33814,154223,0,0,0
02807,189,382,3889,42693,0,0,0
02808,27,116,938,4414,0,0,0


In [45]:
# ZCTA count
# RI has 81 ZCTAs in 2020, but two 02823 and 02836 had no biz establishments in 2021 / 2022 ZBP
zctaemp.shape

(79, 7)

In [47]:
#These are point-based ZIPs that were aggregated to a ZCTA
ptzips=list(zbpemp2zcta.index.difference(zctaemp.index))
zbpemp2zcta.index.difference(zctaemp.index)

Index(['02801', '02862', '02880', '02883', '02887', '02901', '02902', '02940'], dtype='object')

In [49]:
#Double check and make sure that's what these are
#Confirmed for RI that these are point-based and have no area
statezips.loc[statezips.index.isin(ptzips)]

,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type
ZIP_CODE,,,,,
02801,Adamsville,RI,Post Office or large volume customer,02837,Spatial join to ZCTA
02862,Pawtucket,RI,Post Office or large volume customer,02860,Spatial join to ZCTA
02880,Wakefield,RI,Post Office or large volume customer,02879,Spatial join to ZCTA
02883,Peace Dale,RI,Post Office or large volume customer,02879,Spatial join to ZCTA
02887,Warwick,RI,Post Office or large volume customer,02886,Spatial join to ZCTA
02901,Providence,RI,Post Office or large volume customer,02903,Spatial join to ZCTA
02902,Providence,RI,Post Office or large volume customer,02903,Spatial join to ZCTA
02940,Providence,RI,Post Office or large volume customer,02904,Spatial join to ZCTA


In [51]:
# Create a long GEOID column and reset index
zctaemp.insert(loc=0, column='GEOIDLONG',value='860Z200US'+zctaemp.index)
zctaemp=zctaemp.reset_index().set_index('GEOIDLONG')
zctaemp

,zcta,ESTAB,EMP,PAYQ1,PAYAN,FLAG_EMP,FLAG_PAYQ1,FLAG_PAYAN
GEOIDLONG,,,,,,,,
860Z200US02802,02802,9,24,198,1747,0,0,0
860Z200US02804,02804,83,689,5872,28243,0,0,0
860Z200US02806,02806,361,4069,33814,154223,0,0,0
860Z200US02807,02807,189,382,3889,42693,0,0,0
860Z200US02808,02808,27,116,938,4414,0,0,0
...,...,...,...,...,...,...,...,...
860Z200US02917,02917,602,12744,271941,1126104,0,0,0
860Z200US02918,02918,17,2400,24174,103340,0,0,0
860Z200US02919,02919,1049,15285,299427,1059369,0,0,0


### ZBP Industry Data
This data must be requested one record at a time. NOTE that this request can take a LONG TIME, up to 20 minutes for approx 100 ZIP Codes. Once the request is finished the data gets dumped into a json file. If the request is successful but subsequent blocks need to be modified, don't rerun the requests block - pull the data from the json file.

In [54]:
zipcodes=zbpemp2zcta.index.tolist()
len(zipcodes)

87

# ***THIS BLOCK IS A REQUESTS BLOCK!***  
Retrieving approx 100 ZIP Codes takes 20 minutes

*NOTE - revise in the future to retrieve chunks of zip codes*

In [57]:
#If this block is successful but there are subsequent problems, do not rerun it - start from the following block.
#For industry data, if there are no records for an industry create a blank record with zeros
n=0
z=0
ind_data=[['estab','naics','zipcode']]
for zcode in zipcodes:
#for zcode in zipcodes[0:5]:
    clear_output(wait=True)
    for naics in ncodes:
        idata_url = f'{base_url}?get={icols}&NAICS2017={naics}&for=zip code:{zcode}&key={api_key}'
        try:
            response=requests.get(idata_url)
        except requests.exceptions.RequestException as e:
            print (e)
            break
        if response.status_code==200:
            jsondata=response.json()
            ind_data.append(jsondata[1]) 
            n=n+1
        elif response.status_code==204:
            record=['0',naics,zcode]
            ind_data.append(record)
            n=n+1
        else:
            print('Problem retrieving data, status code:',response.status_code)
            break
    z=z+1
    print(n,'records have been retrieved for',z,'ZIP codes...')
    time.sleep(2)
print('Done')

with open(ijsonpath, 'w') as f:
    json.dump(ind_data, f)
print('Data dumped to json file')

1808 records have been retrieved for 87 ZIP codes...
Done
Data dumped to json file


In [59]:
with open(ijsonpath, 'r') as f:
    ijsondata=json.load(f)
zbpind = pd.DataFrame(ijsondata[1:],columns=ijsondata[0])
zbpind['estab']=zbpind['estab'].astype('int64')
zbpind.head()

,estab,naics,zipcode
0,6,00,02801
1,0,11,02801
2,0,21,02801
3,0,22,02801
4,0,23,02801


In [61]:
#Pivot data to move NAICS to columns
zbpind_tab=zbpind.pivot(index='zipcode', columns='naics', values='estab')
zbpind_tab=zbpind_tab.add_prefix('N')
zbpind_tab.rename(columns=lambda x: x.replace('-', '_'),inplace=True)
zbpind_tab.head()

naics,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N53,N54,N55,N56,N61,N62,N71,N72,N81,N99
zipcode,,,,,,,,,,,,,,,,,,,,,
02801,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02804,83.0,0.0,0.0,0.0,22.0,3.0,8.0,7.0,0.0,0.0,...,0.0,7.0,0.0,9.0,0.0,3.0,0.0,8.0,10.0,0.0
02806,361.0,0.0,0.0,0.0,47.0,4.0,8.0,28.0,0.0,9.0,...,26.0,53.0,0.0,21.0,10.0,53.0,17.0,24.0,32.0,0.0
02807,189.0,0.0,0.0,0.0,30.0,4.0,0.0,34.0,5.0,0.0,...,10.0,3.0,0.0,16.0,0.0,0.0,6.0,59.0,14.0,0.0


In [63]:
#Create column to summarize businesses that were not disclosed and categorized, but that are included in the total
zbpind_tab['NXX']=zbpind_tab.loc[:,'N00'].subtract(zbpind_tab.loc[:,'N11':'N99'].sum(axis=1))
zbpind_tab.head()

naics,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N54,N55,N56,N61,N62,N71,N72,N81,N99,NXX
zipcode,,,,,,,,,,,,,,,,,,,,,
02801,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
02804,83.0,0.0,0.0,0.0,22.0,3.0,8.0,7.0,0.0,0.0,...,7.0,0.0,9.0,0.0,3.0,0.0,8.0,10.0,0.0,6.0
02806,361.0,0.0,0.0,0.0,47.0,4.0,8.0,28.0,0.0,9.0,...,53.0,0.0,21.0,10.0,53.0,17.0,24.0,32.0,0.0,4.0
02807,189.0,0.0,0.0,0.0,30.0,4.0,0.0,34.0,5.0,0.0,...,3.0,0.0,16.0,0.0,0.0,6.0,59.0,14.0,0.0,8.0


In [65]:
#Join to ZIP ZCTA crosswalk
zbpind2zcta = pd.merge(statezips[['zcta']],zbpind_tab,how='inner',left_index=True,right_index=True)
zbpind2zcta.index.name = 'ZIP_CODE'
zbpind2zcta.head()

,zcta,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,...,N54,N55,N56,N61,N62,N71,N72,N81,N99,NXX
ZIP_CODE,,,,,,,,,,,,,,,,,,,,,
02801,02837,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
02802,02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
02804,02804,83.0,0.0,0.0,0.0,22.0,3.0,8.0,7.0,0.0,...,7.0,0.0,9.0,0.0,3.0,0.0,8.0,10.0,0.0,6.0
02806,02806,361.0,0.0,0.0,0.0,47.0,4.0,8.0,28.0,0.0,...,53.0,0.0,21.0,10.0,53.0,17.0,24.0,32.0,0.0,4.0
02807,02807,189.0,0.0,0.0,0.0,30.0,4.0,0.0,34.0,5.0,...,3.0,0.0,16.0,0.0,0.0,6.0,59.0,14.0,0.0,8.0


In [67]:
zbpind2zcta.shape

(87, 23)

In [69]:
#Aggregate to ZCTAs
zctaind=zbpind2zcta.groupby(['zcta']).sum()
zctaind.head()

,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N54,N55,N56,N61,N62,N71,N72,N81,N99,NXX
zcta,,,,,,,,,,,,,,,,,,,,,
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
02804,83.0,0.0,0.0,0.0,22.0,3.0,8.0,7.0,0.0,0.0,...,7.0,0.0,9.0,0.0,3.0,0.0,8.0,10.0,0.0,6.0
02806,361.0,0.0,0.0,0.0,47.0,4.0,8.0,28.0,0.0,9.0,...,53.0,0.0,21.0,10.0,53.0,17.0,24.0,32.0,0.0,4.0
02807,189.0,0.0,0.0,0.0,30.0,4.0,0.0,34.0,5.0,0.0,...,3.0,0.0,16.0,0.0,0.0,6.0,59.0,14.0,0.0,8.0
02808,27.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,8.0,0.0,7.0


In [71]:
zctaind.shape

(79, 22)

In [73]:
#Generate and calculate percent total columns
ncols=list(zctaind)
for c in ncols[1:]:
    pct=c+'_PCT'
    zctaind[pct]=((zctaind[c]/zctaind['N00'])*100).round(2)
zctaind.head()

,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N54_PCT,N55_PCT,N56_PCT,N61_PCT,N62_PCT,N71_PCT,N72_PCT,N81_PCT,N99_PCT,NXX_PCT
zcta,,,,,,,,,,,,,,,,,,,,,
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,100.00
02804,83.0,0.0,0.0,0.0,22.0,3.0,8.0,7.0,0.0,0.0,...,8.43,0.0,10.84,0.00,3.61,0.00,9.64,12.05,0.0,7.23
02806,361.0,0.0,0.0,0.0,47.0,4.0,8.0,28.0,0.0,9.0,...,14.68,0.0,5.82,2.77,14.68,4.71,6.65,8.86,0.0,1.11
02807,189.0,0.0,0.0,0.0,30.0,4.0,0.0,34.0,5.0,0.0,...,1.59,0.0,8.47,0.00,0.00,3.17,31.22,7.41,0.0,4.23
02808,27.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,...,0.00,0.0,18.52,0.00,0.00,0.00,0.00,29.63,0.0,25.93


In [75]:
# Create a long GEOID column and reset index
zctaind.insert(loc=0, column='GEOIDLONG',value='860Z200US'+zctaind.index)
zctaind=zctaind.reset_index().set_index('GEOIDLONG')
zctaind

,zcta,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,...,N54_PCT,N55_PCT,N56_PCT,N61_PCT,N62_PCT,N71_PCT,N72_PCT,N81_PCT,N99_PCT,NXX_PCT
GEOIDLONG,,,,,,,,,,,,,,,,,,,,,
860Z200US02802,02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,100.00
860Z200US02804,02804,83.0,0.0,0.0,0.0,22.0,3.0,8.0,7.0,0.0,...,8.43,0.00,10.84,0.00,3.61,0.00,9.64,12.05,0.0,7.23
860Z200US02806,02806,361.0,0.0,0.0,0.0,47.0,4.0,8.0,28.0,0.0,...,14.68,0.00,5.82,2.77,14.68,4.71,6.65,8.86,0.0,1.11
860Z200US02807,02807,189.0,0.0,0.0,0.0,30.0,4.0,0.0,34.0,5.0,...,1.59,0.00,8.47,0.00,0.00,3.17,31.22,7.41,0.0,4.23
860Z200US02808,02808,27.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,...,0.00,0.00,18.52,0.00,0.00,0.00,0.00,29.63,0.0,25.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860Z200US02917,02917,602.0,0.0,0.0,0.0,73.0,37.0,25.0,75.0,12.0,...,8.14,1.33,7.64,1.33,8.14,2.16,11.79,8.80,0.0,0.17
860Z200US02918,02918,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,52.94,0.00,0.0,47.06
860Z200US02919,02919,1049.0,0.0,0.0,5.0,173.0,58.0,31.0,123.0,47.0,...,5.05,0.29,10.68,1.24,10.39,1.43,9.06,9.53,0.0,0.10


### NAICS Codes

#### ***THIS BLOCK IS A REQUESTS BLOCK!***

In [79]:
codedict={}
codes_url=f'https://api.census.gov/data/2022/cbp/variables/NAICS2017.json'
response=requests.get(codes_url)
codes_data=response.json()
codedict.update(codes_data['values']['item'])
sectordict=dict((k, codedict[k]) for k in ncodes)
sectordict['XX']='Establishments omitted from classification due to privacy regulations'

In [81]:
codes=pd.DataFrame(list(sectordict.items()), columns=['naics', 'name']).set_index('naics')
codes

,name
naics,
00,Total for all sectors
11,"Agriculture, forestry, fishing and hunting"
21,"Mining, quarrying, and oil and gas extraction"
22,Utilities
23,Construction
31-33,Manufacturing
42,Wholesale trade
44-45,Retail trade
48-49,Transportation and warehousing


In [83]:
# Remove key from memory
api_key=None

## Quality Control Checks

In [86]:
#Does sum of industries equal industry total?
indsum=zctaind['N00'].subtract(zctaind.iloc[:,2:23].sum(axis=1))
if indsum.sum()==0:
    print (True)
else:
    print(indsum.loc[indsum != 0])

True


In [88]:
#Is sum of percent totals approximately 100?
ptotal=zctaind.iloc[:,23:].sum(axis=1)
if ptotal.loc[(ptotal <= 99.05) | (ptotal >= 100.05)].empty:
    print(True)
else:
    print(ptotal.loc[(ptotal <= 99.05) | (ptotal >= 100.05)])


True


In [90]:
#Do number of ZCTAs in employment table match the industries table?
ecount=zctaemp.shape[0]
icount=zctaind.shape[0]
if ecount == icount:
    print (True)
else:
    print('Mistmatched count between employment',ecount, 'rows and industry',icount, 'rows')

True


In [92]:
#Does sum of estabslishments from employment table equal establishments in industries table?
estsum=zctaemp['ESTAB'].subtract(zctaind['N00'])
if estsum.sum()==0:
    print (True)
else:
    print(estsum.loc[estsum != 0])

True


## Write to Database 

In [95]:
con = sqlite3.connect(dbname) 
cur = con.cursor()

In [97]:
#Employment table
cur.execute('DROP TABLE IF EXISTS {};'.format(emptable))
qcreate_emptab="""
CREATE TABLE {}(
GEOIDLONG TEXT,
ZCTA TEXT NOT NULL PRIMARY KEY,
ESTAB INTEGER,
EMP INTEGER,
PAYQ1 INTEGER,
PAYAN INTEGER,
FLAG_EMP INTEGER,
FLAG_PAYQ1 INTEGER,
FLAG_PAYAN INTEGER);
""".format(emptable)

cur.execute(qcreate_emptab)

In [99]:
#Don't run this block unless you've run the previous one
zctaemp.to_sql(name='{}'.format(emptable), if_exists='append', index=True, con=con)

79

In [101]:
cur.execute('SELECT COUNT(*) FROM {};'.format(emptable))
rows = cur.fetchone()
print(rows[0], 'records written to', emptable)

79 records written to zbp2022_emp


In [103]:
#Replace zeros with nulls, as these values really represent no data
for col in zctaemp.columns[1:]:
    qupdate='UPDATE {} SET {} = NULL WHERE {} = 0;'.format(emptable,col,col)
    cur.execute(qupdate)
    print(cur.rowcount,'records updated for',col)
    con.commit()

0 records updated for ESTAB
0 records updated for EMP
0 records updated for PAYQ1
0 records updated for PAYAN
79 records updated for FLAG_EMP
79 records updated for FLAG_PAYQ1
79 records updated for FLAG_PAYAN


In [105]:
#Industry table
cur.execute('DROP TABLE IF EXISTS {}'.format(indtable))
qcreate_indtab="""
CREATE TABLE {} (
GEOIDLONG TEXT,
ZCTA TEXT NOT NULL PRIMARY KEY, 
N00 INTEGER, 
N11 INTEGER, 
N21 INTEGER, 
N22 INTEGER, 
N23 INTEGER, 
N31_33 INTEGER, 
N42 INTEGER, 
N44_45 INTEGER, 
N48_49 INTEGER, 
N51 INTEGER, 
N52 INTEGER, 
N53 INTEGER, 
N54 INTEGER, 
N55 INTEGER, 
N56 INTEGER, 
N61 INTEGER, 
N62 INTEGER, 
N71 INTEGER, 
N72 INTEGER, 
N81 INTEGER, 
N99 INTEGER,
NXX INTEGER,
N11_PCT REAL, 
N21_PCT REAL, 
N22_PCT REAL, 
N23_PCT REAL, 
N31_33_PCT REAL, 
N42_PCT REAL, 
N44_45_PCT REAL, 
N48_49_PCT REAL, 
N51_PCT REAL, 
N52_PCT REAL, 
N53_PCT REAL, 
N54_PCT REAL, 
N55_PCT REAL, 
N56_PCT REAL, 
N61_PCT REAL, 
N62_PCT REAL, 
N71_PCT REAL, 
N72_PCT REAL, 
N81_PCT REAL, 
N99_PCT REAL,
NXX_PCT REAL);
""".format(indtable)

cur.execute(qcreate_indtab)

In [107]:
#Don't run this block unless you've run the previous one
zctaind.to_sql(name='{}'.format(indtable), if_exists='append', index=True, con=con)

79

In [109]:
cur.execute('SELECT COUNT(*) FROM {};'.format(indtable))
rows = cur.fetchone()
print(rows[0], 'records written to', indtable)

79 records written to zbp2022_ind


In [111]:
#For percentages, replace zeros with nulls, as these values really represent no data
for col in zctaind.columns[22:]:
    qupdate='UPDATE {} SET {} = NULL WHERE {} = 0.0;'.format(indtable,col,col)
    cur.execute(qupdate)
    print(cur.rowcount,'records updated for',col)
    con.commit()

6 records updated for NXX
75 records updated for N11_PCT
79 records updated for N21_PCT
75 records updated for N22_PCT
8 records updated for N23_PCT
24 records updated for N31_33_PCT
23 records updated for N42_PCT
18 records updated for N44_45_PCT
33 records updated for N48_49_PCT
42 records updated for N51_PCT
30 records updated for N52_PCT
24 records updated for N53_PCT
18 records updated for N54_PCT
57 records updated for N55_PCT
17 records updated for N56_PCT
37 records updated for N61_PCT
19 records updated for N62_PCT
28 records updated for N71_PCT
15 records updated for N72_PCT
15 records updated for N81_PCT
79 records updated for N99_PCT
6 records updated for NXX_PCT


In [113]:
#For establishments, replace zeros with nulls unless establishments were omitted from classification
for col in zctaind.columns[1:22]:
    qupdate='UPDATE {} SET {} = NULL WHERE {} = 0 AND NXX !=0;'.format(indtable,col,col)
    cur.execute(qupdate)
    print(cur.rowcount,'records updated for',col)
    con.commit()

0 records updated for N00
70 records updated for N11
73 records updated for N21
70 records updated for N22
8 records updated for N23
23 records updated for N31_33
22 records updated for N42
17 records updated for N44_45
32 records updated for N48_49
40 records updated for N51
29 records updated for N52
23 records updated for N53
17 records updated for N54
55 records updated for N55
16 records updated for N56
36 records updated for N61
18 records updated for N62
27 records updated for N71
14 records updated for N72
14 records updated for N81
73 records updated for N99


In [115]:
#NAICS code table
cur.execute('DROP TABLE IF EXISTS {};'.format(codetable))
qcreate_codetab="""
CREATE TABLE {}(
naics TEXT NOT NULL PRIMARY KEY,
name TEXT);
""".format(codetable)

cur.execute(qcreate_codetab)

In [117]:
#Don't run this block unless you've run the previous one
codes.to_sql(name='{}'.format(codetable), if_exists='append', index=True, con=con)

22

In [119]:
cur.execute('SELECT COUNT(*) FROM {};'.format(codetable))
rows = cur.fetchone()
print(rows[0], 'records written to', codetable)

22 records written to zbp2022_indcodes


In [121]:
#Crosswalk table
cur.execute('DROP TABLE IF EXISTS {};'.format(cwalktable))
qcreate_z2ztab="""
CREATE TABLE {}(
ZIP_CODE TEXT NOT NULL PRIMARY KEY,
PO_NAME TEXT,
STATE TEXT,
ZIP_TYPE TEXT,
zcta TEXT,
zip_join_type TEXT);
""".format(cwalktable)

cur.execute(qcreate_z2ztab)

In [123]:
#Don't run this block unless you've run the previous one
statezips.to_sql(name='{}'.format(cwalktable), if_exists='append', index=True, con=con)

90

In [125]:
cur.execute('SELECT COUNT(*) FROM {};'.format(cwalktable))
rows = cur.fetchone()
print(rows[0], 'records written to', cwalktable)

90 records written to zip2zcta_2022


In [127]:
con.close()